Seeley McGillis

Homework 2: Two ways with MIMIC-III


---

# Part 1: Relational

In [16]:
!pip install duckdb==1.2.2
import duckdb

In [18]:
# establish connection
conn = duckdb.connect('mimic.db', read_only=False)

#conn.execute("""
#CREATE TABLE PRESCRIPTIONS AS
#SELECT * FROM read_csv_auto('PRESCRIPTIONS.csv')
#""")

conn.sql('SHOW TABLES;')

┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ ADMISSIONS    │
│ DRGCODES      │
│ DRUGS_ETHN    │
│ D_ICDPROCS    │
│ ICUSTAYS      │
│ PATIENTS      │
│ PRESCRIPTIONS │
│ PROCS_ICD     │
└───────────────┘

-------------

Create a summary of type of drugs and their total amount used by ethnicity. Report the top usage in each ethnicity group. You may have to make certain assumptions in calculating their total amount.

In [22]:
conn.sql('PRAGMA table_info(PATIENTS);')

┌───────┬─────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │    name     │  type   │ notnull │ dflt_value │   pk    │
│ int32 │   varchar   │ varchar │ boolean │  varchar   │ boolean │
├───────┼─────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ row_id      │ INTEGER │ false   │ NULL       │ false   │
│     1 │ subject_id  │ INTEGER │ false   │ NULL       │ false   │
│     2 │ gender      │ VARCHAR │ false   │ NULL       │ false   │
│     3 │ dob         │ VARCHAR │ false   │ NULL       │ false   │
│     4 │ dod         │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ dod_hosp    │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ dod_ssn     │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ expire_flag │ INTEGER │ false   │ NULL       │ false   │
└───────┴─────────────┴─────────┴─────────┴────────────┴─────────┘

In [24]:
conn.sql('PRAGMA table_info(ADMISSIONS);')

┌───────┬──────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │         name         │  type   │ notnull │ dflt_value │   pk    │
│ int32 │       varchar        │ varchar │ boolean │  varchar   │ boolean │
├───────┼──────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ row_id               │ INTEGER │ false   │ NULL       │ false   │
│     1 │ subject_id           │ INTEGER │ false   │ NULL       │ false   │
│     2 │ hadm_id              │ INTEGER │ false   │ NULL       │ false   │
│     3 │ admittime            │ VARCHAR │ false   │ NULL       │ false   │
│     4 │ dischtime            │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ deathtime            │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ admission_type       │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ admission_location   │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ discharge_location   │ VARCHAR │ false   │ NULL       │ false   │
│     9 │ in

In [26]:
conn.sql(
    """
    CREATE OR REPLACE TABLE DRUGS_ETHN AS
    SELECT
        ADMISSIONS.ETHNICITY,
        PRESCRIPTIONS.DRUG,
        COUNT(PRESCRIPTIONS.DRUG) AS TOTAL_DRUG
    FROM ADMISSIONS
    JOIN PRESCRIPTIONS ON ADMISSIONS.SUBJECT_ID = PRESCRIPTIONS.SUBJECT_ID
    GROUP BY ADMISSIONS.ETHNICITY, PRESCRIPTIONS.DRUG;
    """
)

In [28]:
conn.sql(
    """
    SELECT * FROM DRUGS_ETHN;
    """
)

┌──────────────────────────────────────────────────────────┬──────────────────────┬────────────┐
│                        ethnicity                         │         drug         │ TOTAL_DRUG │
│                         varchar                          │       varchar        │   int64    │
├──────────────────────────────────────────────────────────┼──────────────────────┼────────────┤
│ WHITE                                                    │ Senna                │         95 │
│ WHITE                                                    │ Aspirin              │         50 │
│ WHITE                                                    │ Tamsulosin           │          8 │
│ WHITE                                                    │ Azithromycin         │         21 │
│ WHITE                                                    │ Furosemide           │        388 │
│ WHITE                                                    │ Atropine Sulfate     │          6 │
│ WHITE                       

In [30]:
conn.sql(
    """
    SELECT ETHNICITY, DRUG, TOTAL_DRUG
    FROM DRUGS_ETHN
    WHERE (ETHNICITY, TOTAL_DRUG) IN (SELECT ETHNICITY, MAX(TOTAL_DRUG) FROM DRUGS_ETHN GROUP BY ETHNICITY)
    ORDER BY ETHNICITY
    """
)

┌──────────────────────────────────────────────────────────┬──────────────────────┬────────────┐
│                        ethnicity                         │         drug         │ TOTAL_DRUG │
│                         varchar                          │       varchar        │   int64    │
├──────────────────────────────────────────────────────────┼──────────────────────┼────────────┤
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ 5% Dextrose          │         54 │
│ ASIAN                                                    │ D5W                  │         27 │
│ BLACK/AFRICAN AMERICAN                                   │ Insulin              │         60 │
│ HISPANIC OR LATINO                                       │ 5% Dextrose          │         28 │
│ HISPANIC/LATINO - PUERTO RICAN                           │ 0.9% Sodium Chloride │       1290 │
│ OTHER                                                    │ NS                   │         11 │
│ UNABLE TO OBTAIN            

-----

Create a summary of procedures performed on patients by age groups (<=19, 20-49, 50-79, >80). Report the top three procedures, along with the name of the procedures, performed in each age group.

In [34]:
conn.sql('PRAGMA table_info(PROCS_ICD);')

┌───────┬────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │    name    │  type   │ notnull │ dflt_value │   pk    │
│ int32 │  varchar   │ varchar │ boolean │  varchar   │ boolean │
├───────┼────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ row_id     │ INTEGER │ false   │ NULL       │ false   │
│     1 │ subject_id │ INTEGER │ false   │ NULL       │ false   │
│     2 │ hadm_id    │ INTEGER │ false   │ NULL       │ false   │
│     3 │ seq_num    │ INTEGER │ false   │ NULL       │ false   │
│     4 │ icd9_code  │ INTEGER │ false   │ NULL       │ false   │
└───────┴────────────┴─────────┴─────────┴────────────┴─────────┘

In [36]:
conn.sql('PRAGMA table_info(D_ICDPROCS);')

┌───────┬─────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │    name     │  type   │ notnull │ dflt_value │   pk    │
│ int32 │   varchar   │ varchar │ boolean │  varchar   │ boolean │
├───────┼─────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ row_id      │ INTEGER │ false   │ NULL       │ false   │
│     1 │ icd9_code   │ INTEGER │ false   │ NULL       │ false   │
│     2 │ short_title │ VARCHAR │ false   │ NULL       │ false   │
│     3 │ long_title  │ VARCHAR │ false   │ NULL       │ false   │
└───────┴─────────────┴─────────┴─────────┴────────────┴─────────┘

In [38]:
conn.sql(
    """
    SELECT DOB FROM PATIENTS
    """
)

┌─────────────────────┐
│         dob         │
│       varchar       │
├─────────────────────┤
│ 2094-03-05 00:00:00 │
│ 2090-06-05 00:00:00 │
│ 2038-09-03 00:00:00 │
│ 2075-09-21 00:00:00 │
│ 2114-06-20 00:00:00 │
│ 1895-05-17 00:00:00 │
│ 2108-01-15 00:00:00 │
│ 2061-04-10 00:00:00 │
│ 2050-03-29 00:00:00 │
│ 2051-04-21 00:00:00 │
│          ·          │
│          ·          │
│          ·          │
│ 2097-05-16 00:00:00 │
│ 2051-07-25 00:00:00 │
│ 2051-03-24 00:00:00 │
│ 2073-11-22 00:00:00 │
│ 2104-02-12 00:00:00 │
│ 2057-11-15 00:00:00 │
│ 1878-05-14 00:00:00 │
│ 2078-06-16 00:00:00 │
│ 2107-06-27 00:00:00 │
│ 2112-10-22 00:00:00 │
├─────────────────────┤
│ 100 rows (20 shown) │
└─────────────────────┘

In [40]:
conn.sql(
    """
    CREATE OR REPLACE TABLE PROC_AGE AS
    SELECT
        CASE
            WHEN CAST(STRFTIME('%Y', CAST(ADMISSIONS.ADMITTIME AS DATE)) AS INT) - CAST(STRFTIME('%Y', CAST(PATIENTS.DOB AS DATE)) AS INT) <= 19 THEN '<=19'
            WHEN CAST(STRFTIME('%Y', CAST(ADMISSIONS.ADMITTIME AS DATE)) AS INT) - CAST(STRFTIME('%Y', CAST(PATIENTS.DOB AS DATE)) AS INT) BETWEEN 20 AND 49 THEN '20-49'
            WHEN CAST(STRFTIME('%Y', CAST(ADMISSIONS.ADMITTIME AS DATE)) AS INT) - CAST(STRFTIME('%Y', CAST(PATIENTS.DOB AS DATE)) AS INT) BETWEEN 50 AND 79 THEN '50-79'
            ELSE '>80'
        END AS AGE_GROUP,
        PROCS_ICD.ICD9_CODE,
        COUNT(PROCS_ICD.ICD9_CODE) AS TOTAL_PROC
    FROM ADMISSIONS
    JOIN PATIENTS ON ADMISSIONS.SUBJECT_ID = PATIENTS.SUBJECT_ID
    JOIN PROCS_ICD ON ADMISSIONS.HADM_ID = PROCS_ICD.HADM_ID
    GROUP BY AGE_GROUP, PROCS_ICD.ICD9_CODE;
    """
)

In [42]:
#Display table
conn.sql(
    """
    SELECT * FROM PROC_AGE;
    """
)

┌───────────┬───────────┬────────────┐
│ AGE_GROUP │ icd9_code │ TOTAL_PROC │
│  varchar  │   int32   │   int64    │
├───────────┼───────────┼────────────┤
│ 50-79     │      8345 │          1 │
│ 50-79     │      9915 │          8 │
│ >80       │      3772 │          2 │
│ >80       │      9962 │          3 │
│ 50-79     │      9929 │          1 │
│ <=19      │      7935 │          1 │
│ <=19      │      7905 │          1 │
│ <=19      │      3893 │          2 │
│ 20-49     │      3404 │          2 │
│ 20-49     │      3712 │          1 │
│   ·       │        ·  │          · │
│   ·       │        ·  │          · │
│   ·       │        ·  │          · │
│ 20-49     │      3891 │          1 │
│ 20-49     │      5187 │          1 │
│ 20-49     │      3990 │          1 │
│ 20-49     │      9671 │          4 │
│ >80       │      3324 │          2 │
│ 50-79     │      5110 │          1 │
│ 20-49     │      3895 │          2 │
│ 50-79     │      9390 │          1 │
│ 50-79     │       311 │

In [44]:
#Report the top three procedures performed in <=19 age group
conn.sql(
    """
    CREATE OR REPLACE TABLE PROC_AGE_1 AS
    SELECT AGE_GROUP, D_ICDPROCS.SHORT_TITLE, TOTAL_PROC
    FROM PROC_AGE
    JOIN D_ICDPROCS ON PROC_AGE.ICD9_CODE = D_ICDPROCS.ICD9_CODE
    WHERE AGE_GROUP = '<=19'
    ORDER BY TOTAL_PROC DESC
    LIMIT 3
    """
)

In [46]:
#Display table
conn.sql(
    """
    SELECT * FROM PROC_AGE_1;
    """
)

┌───────────┬─────────────────────────┬────────────┐
│ AGE_GROUP │       short_title       │ TOTAL_PROC │
│  varchar  │         varchar         │   int64    │
├───────────┼─────────────────────────┼────────────┤
│ <=19      │ Venous cath NEC         │          2 │
│ <=19      │ Closed bronchial biopsy │          1 │
│ <=19      │ Vertebral fx repair     │          1 │
└───────────┴─────────────────────────┴────────────┘

In [48]:
#Report the top three procedures performed in 20-49 age group
conn.sql(
    """
    CREATE OR REPLACE TABLE PROC_AGE_2 AS
    SELECT AGE_GROUP, D_ICDPROCS.SHORT_TITLE, TOTAL_PROC
    FROM PROC_AGE
    JOIN D_ICDPROCS ON PROC_AGE.ICD9_CODE = D_ICDPROCS.ICD9_CODE
    WHERE AGE_GROUP = '20-49'
    ORDER BY TOTAL_PROC DESC
    LIMIT 3
    """
)

In [50]:
#Display table
conn.sql(
    """
    SELECT * FROM PROC_AGE_2;
    """
)

┌───────────┬──────────────────────────┬────────────┐
│ AGE_GROUP │       short_title        │ TOTAL_PROC │
│  varchar  │         varchar          │   int64    │
├───────────┼──────────────────────────┼────────────┤
│ 20-49     │ Venous cath NEC          │          9 │
│ 20-49     │ Entral infus nutrit sub  │          7 │
│ 20-49     │ Insert endotracheal tube │          6 │
└───────────┴──────────────────────────┴────────────┘

In [52]:
#Report the top three procedures performed in 50-79 age group
conn.sql(
    """
    CREATE OR REPLACE TABLE PROC_AGE_3 AS
    SELECT AGE_GROUP, D_ICDPROCS.SHORT_TITLE, TOTAL_PROC
    FROM PROC_AGE
    JOIN D_ICDPROCS ON PROC_AGE.ICD9_CODE = D_ICDPROCS.ICD9_CODE
    WHERE AGE_GROUP = '50-79'
    ORDER BY TOTAL_PROC DESC
    LIMIT 3
    """
)

In [54]:
#Display table
conn.sql(
    """
    SELECT * FROM PROC_AGE_3;
    """
)

┌───────────┬─────────────────────────┬────────────┐
│ AGE_GROUP │       short_title       │ TOTAL_PROC │
│  varchar  │         varchar         │   int64    │
├───────────┼─────────────────────────┼────────────┤
│ 50-79     │ Venous cath NEC         │         25 │
│ 50-79     │ Entral infus nutrit sub │         22 │
│ 50-79     │ Packed cell transfusion │         13 │
└───────────┴─────────────────────────┴────────────┘

In [56]:
#Report the top three procedures performed in >80 age group
conn.sql(
    """
    CREATE OR REPLACE TABLE PROC_AGE_4 AS
    SELECT AGE_GROUP, D_ICDPROCS.SHORT_TITLE, TOTAL_PROC
    FROM PROC_AGE
    JOIN D_ICDPROCS ON PROC_AGE.ICD9_CODE = D_ICDPROCS.ICD9_CODE
    WHERE AGE_GROUP = '>80'
    ORDER BY TOTAL_PROC DESC
    LIMIT 3
    """
)

In [58]:
#Display table
conn.sql(
    """
    SELECT * FROM PROC_AGE_4;
    """
)

┌───────────┬──────────────────────────┬────────────┐
│ AGE_GROUP │       short_title        │ TOTAL_PROC │
│  varchar  │         varchar          │   int64    │
├───────────┼──────────────────────────┼────────────┤
│ >80       │ Venous cath NEC          │         20 │
│ >80       │ Packed cell transfusion  │         13 │
│ >80       │ Insert endotracheal tube │          8 │
└───────────┴──────────────────────────┴────────────┘

-----

How long do patients stay in the ICU? Is there a difference in the ICU length of stay among gender or ethnicity?

In [62]:
conn.sql('PRAGMA table_info(ICUSTAYS);')

┌───────┬────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │      name      │  type   │ notnull │ dflt_value │   pk    │
│ int32 │    varchar     │ varchar │ boolean │  varchar   │ boolean │
├───────┼────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ row_id         │ INTEGER │ false   │ NULL       │ false   │
│     1 │ subject_id     │ INTEGER │ false   │ NULL       │ false   │
│     2 │ hadm_id        │ INTEGER │ false   │ NULL       │ false   │
│     3 │ icustay_id     │ INTEGER │ false   │ NULL       │ false   │
│     4 │ dbsource       │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ first_careunit │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ last_careunit  │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ first_wardid   │ INTEGER │ false   │ NULL       │ false   │
│     8 │ last_wardid    │ INTEGER │ false   │ NULL       │ false   │
│     9 │ intime         │ VARCHAR │ false   │ NULL       │ false   │
│    10 │ outtime   

In [64]:
conn.sql(
    """
    SELECT AVG(ICUSTAYS.LOS) AS AVG_LOS
    FROM ICUSTAYS;
    """
)

┌───────────────────┐
│      AVG_LOS      │
│      double       │
├───────────────────┤
│ 4.452456642106614 │
└───────────────────┘

In [66]:
conn.sql(
    """
    CREATE OR REPLACE TABLE ICU_GENDER AS
    SELECT PATIENTS.GENDER, AVG(ICUSTAYS.LOS) AS AVG_LOS
    FROM ICUSTAYS
    JOIN PATIENTS ON ICUSTAYS.SUBJECT_ID = PATIENTS.SUBJECT_ID
    GROUP BY PATIENTS.GENDER;
    """
)

In [68]:
#Display table
conn.sql(
    """
    SELECT * FROM ICU_GENDER;
    """
)

┌─────────┬────────────────────┐
│ gender  │      AVG_LOS       │
│ varchar │       double       │
├─────────┼────────────────────┤
│ F       │  5.540071457151383 │
│ M       │ 3.5138301578898954 │
└─────────┴────────────────────┘

In [70]:
conn.sql(
    """
    CREATE OR REPLACE TABLE ICU_ETHN AS
    SELECT ADMISSIONS.ETHNICITY, AVG(ICUSTAYS.LOS) AS AVG_LOS
    FROM ICUSTAYS
    JOIN ADMISSIONS ON ICUSTAYS.SUBJECT_ID = ADMISSIONS.SUBJECT_ID
    GROUP BY ADMISSIONS.ETHNICITY;
    """
)

In [180]:
#Display table
conn.sql(
    """
    SELECT * FROM ICU_ETHN;
    """
)

┌──────────────────────────────────────────────────────────┬────────────────────┐
│                        ethnicity                         │      AVG_LOS       │
│                         varchar                          │       double       │
├──────────────────────────────────────────────────────────┼────────────────────┤
│ OTHER                                                    │ 0.9260666593909264 │
│ WHITE                                                    │  4.123054919062091 │
│ UNKNOWN/NOT SPECIFIED                                    │  4.510661611190209 │
│ ASIAN                                                    │  3.890050023794174 │
│ BLACK/AFRICAN AMERICAN                                   │  6.867700040340424 │
│ HISPANIC OR LATINO                                       │  7.459633429845174 │
│ UNABLE TO OBTAIN                                         │ 13.357000350952148 │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │  11.33715045452118 │
│ HISPANIC/LATIN

------------------

# Part 2: Non Relational

Setup:

In [77]:
!curl https://certs.secureserver.net/repository/sf-class2-root.crt -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1468  100  1468    0     0   3224      0 --:--:-- --:--:-- --:--:--  3219


In [79]:
%pip install cassandra-sigv4

Note: you may need to restart the kernel to use updated packages.


In [81]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

/var/folders/wd/g730h79n6g90zsm1jp44dyv00000gn/T/ipykernel_89890/332746716.py:7: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2)


In [82]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [83]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

In [84]:
# establishing connection to Keyspace
session = cluster.connect()

In [88]:
# Insert any CQL queries between .connect() and .shutdown()

# For example, show all keyspaces created
r = session.execute('''
    SELECT * FROM system_schema.keyspaces;
    ''')
print(r.current_rows)

[Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_schema_mcs', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_multiregion_info', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='de300_axc4282', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='de300_demo', durable_writes=True, replication=OrderedMapSe

In [89]:
# For example, create a keyspace for HW2
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS seeley_hw2
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

[]


In [91]:
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel


# Define execution profile with LOCAL_QUORUM
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

# Cluster setup with correct profile
cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

# establishing connection to Keyspace
session = cluster.connect()
session.set_keyspace('seeley_hw2')  # Replace with your keyspace

-------------

Question 1:

In [96]:
session.execute("""
    CREATE TABLE IF NOT EXISTS DRUGS_ETHN (
        ethnicity TEXT,
        drug TEXT,
        total_drug INT,
        PRIMARY KEY (ethnicity, drug)
    )
""")

In [105]:
drugs_ethn_df = conn.execute("SELECT * FROM DRUGS_ETHN").fetchdf()

print(drugs_ethn_df.columns)

Index(['ethnicity', 'drug', 'TOTAL_DRUG'], dtype='object')


In [107]:
insert_stmt_1 = session.prepare("""
    INSERT INTO drugs_ethn (ethnicity, drug, total_drug)
    VALUES (?, ?, ?)
""")

for row in drugs_ethn_df.itertuples(index=False):
    session.execute(insert_stmt_1, (row.ethnicity, row.drug, int(row.TOTAL_DRUG)))

In [108]:
import csv

rows = session.execute("SELECT * FROM drugs_ethn")
with open("drugs_ethn.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ethnicity", "drug", "total_drug"])
    for row in rows:
        writer.writerow([row.ethnicity, row.drug, int(row.total_drug)])

--------------

Question 2:

In [119]:
session.execute("""
    CREATE TABLE IF NOT EXISTS PROC_AGE (
        AGE_GROUP TEXT,
        icd9_code INT,
        TOTAL_PROC INT,
        PRIMARY KEY (AGE_GROUP, icd9_code)
    )
""")

In [121]:
proc_age_df = conn.execute("SELECT * FROM PROC_AGE").fetchdf()

In [123]:
insert_stmt_2 = session.prepare("""
    INSERT INTO PROC_AGE (AGE_GROUP, icd9_code, TOTAL_PROC)
    VALUES (?, ?, ?)
""")

for row in proc_age_df.itertuples(index=False):
    session.execute(insert_stmt_2, (row.AGE_GROUP, row.icd9_code, row.TOTAL_PROC))

In [128]:
rows = session.execute("SELECT * FROM PROC_AGE")
with open("proc_age.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["AGE_GROUP", "icd9_code", "TOTAL_PROC"])
    for row in rows:
        writer.writerow([row.age_group, row.icd9_code, row.total_proc])

In [130]:
session.execute("""
    CREATE TABLE IF NOT EXISTS TOP_PROC_AGE (
        AGE_GROUP TEXT,
        short_title TEXT,
        TOTAL_PROC INT,
        PRIMARY KEY (AGE_GROUP, short_title)
    )
""")

In [143]:
proc_age_df_1 = conn.execute("SELECT * FROM PROC_AGE_1").fetchdf()
proc_age_df_2 = conn.execute("SELECT * FROM PROC_AGE_2").fetchdf()
proc_age_df_3 = conn.execute("SELECT * FROM PROC_AGE_3").fetchdf()
proc_age_df_4 = conn.execute("SELECT * FROM PROC_AGE_4").fetchdf()

In [145]:
insert_stmt_3 = session.prepare("""
    INSERT INTO TOP_PROC_AGE (AGE_GROUP, short_title, TOTAL_PROC)
    VALUES (?, ?, ?)
""")

for row in proc_age_df_1.itertuples(index=False):
    session.execute(insert_stmt_3, (row.AGE_GROUP, row.short_title, row.TOTAL_PROC))

for row in proc_age_df_2.itertuples(index=False):
    session.execute(insert_stmt_3, (row.AGE_GROUP, row.short_title, row.TOTAL_PROC))

for row in proc_age_df_3.itertuples(index=False):
    session.execute(insert_stmt_3, (row.AGE_GROUP, row.short_title, row.TOTAL_PROC))

for row in proc_age_df_4.itertuples(index=False):
    session.execute(insert_stmt_3, (row.AGE_GROUP, row.short_title, row.TOTAL_PROC))

In [147]:
rows = session.execute("SELECT * FROM TOP_PROC_AGE")
with open("proc_age_top.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["AGE_GROUP", "short_title", "TOTAL_PROC"])
    for row in rows:
        writer.writerow([row.age_group, row.short_title, row.total_proc])

-------------

Question 3:

In [152]:
session.execute("""
    CREATE TABLE IF NOT EXISTS ICU_GENDER (
        gender TEXT,
        AVG_LOS DOUBLE,
        PRIMARY KEY (gender, AVG_LOS)
    )
""")

In [154]:
gender_icu_df = conn.execute("SELECT * FROM ICU_GENDER").fetchdf()

In [156]:
insert_stmt_4 = session.prepare("""
    INSERT INTO ICU_GENDER (gender, AVG_LOS)
    VALUES (?, ?)
""")

for row in gender_icu_df.itertuples(index=False):
    session.execute(insert_stmt_4, (row.gender, row.AVG_LOS))

In [158]:
rows = session.execute("SELECT * FROM ICU_GENDER")
with open("gender_icu.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["gender", "AVG_LOS"])
    for row in rows:
        writer.writerow([row.gender, row.avg_los])

In [160]:
session.execute("""
    CREATE TABLE IF NOT EXISTS ICU_ETHN (
        ethnicity TEXT,
        AVG_LOS DOUBLE,
        PRIMARY KEY (ethnicity, AVG_LOS)
    )
""")

In [186]:
ethn_icu_df = conn.execute("SELECT * FROM ICU_ETHN").fetchdf()

In [188]:
insert_stmt_5 = session.prepare("""
    INSERT INTO ICU_ETHN (ethnicity, AVG_LOS)
    VALUES (?, ?)
""")

for row in ethn_icu_df.itertuples(index=False):
    session.execute(insert_stmt_5, (row.ethnicity, row.AVG_LOS))

In [190]:
rows = session.execute("SELECT * FROM ICU_ETHN")
with open("ethnicity_icu.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ethnicity", "AVG_LOS"])
    for row in rows:
        writer.writerow([row.ethnicity, row.avg_los])

--------